<a href="https://colab.research.google.com/github/mtareqsoliman/msthesis/blob/main/1_OpenSeesAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **1. Obtaining the training dataset of the fragility model**

## 1.1. Installing the required libraries

PyDOE2: for providing sampling sets (Latin Hypercube, Response Surface Method, etc.)

OpenSeesPy: OpenSees library in python for finite element modelling.

Opsvis: OpenSeesPy postprocessing and visualization module, was used in modelling the sections of the structure.

In [1]:
pip install pyDOE2 openseespy opsvis

## 1.2. Generating the training dataset

Here we will generate the sampling dataset that will be used in training the fragility model. The sampling dataset options is with different options.

1. Monte Carlo Simulations.   
2. Central Composite Design (for training Response Surface Method).
3. Grid Sampling (for training ANN).
4. Latin Hypercube Simulations.  

the default here has been set to Monte Carlo Simulations with sampling size 25.

### 1.2.1 Preparing the functions  
(planned to be moved out to a different file)

In [2]:
from pyDOE2 import lhs,ccdesign
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sps

def samples_MCS(nsamples=25):
    design = np.random.standard_normal(size=(nsamples,2))
    return design

def samples_RSM(nsamples=9):
    nsamples = 9 # this is for the function not to produce an error if it is passed with a value.
    design = ccdesign(2,(1,1),'r','ccc')
    return design

def samples_grid(nsamples=25):
    x1_mesh,x2_mesh = np.meshgrid(np.linspace(-2,2,nsamples),np.linspace(-2,2,nsamples))
    design = np.array([x1_mesh.flatten(), x2_mesh.flatten()]).T
    return design

# Latin hypercube
def samples_LHS(nsamples=25):
    design = lhs(2, nsamples)
    for i in np.arange(0,2):
        design[:, i] = sps.norm(loc=0, scale=1).ppf(design[:, i])
    return design

nsamples = 25
sampling_set = samples_MCS(nsamples)
sampling_set

array([[-0.08971576,  0.39910132],
       [-0.04627143, -2.34261962],
       [ 0.41079696, -0.49410998],
       [-1.44252231, -0.64643093],
       [ 0.81360915, -0.88948463],
       [-1.09237165,  0.54900264],
       [-0.98680946, -0.42435926],
       [ 0.08060868, -0.57707687],
       [ 2.03191428, -1.37106101],
       [-1.85238696, -2.48794275],
       [ 1.09849917,  1.03630702],
       [-0.62217515, -1.18033892],
       [-0.91537076, -0.35797486],
       [-1.01976293, -0.3537815 ],
       [ 0.16562129, -2.21443795],
       [-0.04112526, -1.58440016],
       [ 0.5279066 ,  0.40916546],
       [ 0.10475428,  0.96559511],
       [ 1.78580769, -0.16265782],
       [-0.01236084,  0.0625859 ],
       [ 1.12002962,  1.49150205],
       [-0.47932863,  1.29888041],
       [ 0.492555  , -0.14225946],
       [-0.57318343, -0.7213048 ],
       [-1.63311972,  2.05760146]])

### 1.2.2 The main sampling function and sampling size are selected from the dropdown menu and the slider.

In [3]:
import ipywidgets as widgets
from IPython.display import display

# Create sliders
function_dict = ['Monte Carlo Simulations','Response Surface Method','Artificial Neural Networks','Latin Hypercube']
function_dropdown = widgets.Dropdown(options=function_dict, value=function_dict[0], description='Select Function:')
argument_slider = widgets.IntSlider(value=25, min=0, max=200, step=25, description='Select Argument:')

# Map the functions to the first slider values
function_mapping = {'Monte Carlo Simulations':(samples_MCS),'Response Surface Method':(samples_RSM),'Artificial Neural Networks':(samples_grid),'Latin Hypercube':(samples_LHS)}

# Function to be called when sliders change
def apply_function(change):
    selected_function = function_mapping.get(function_dropdown.value, lambda x: None)
    global sampling_set
    sampling_set = selected_function(argument_slider.value)
    print(sampling_set)
    return sampling_set

# Attach the function application to the sliders' 'value' attributes
function_dropdown.observe(apply_function, names='value')
argument_slider.observe(apply_function, names='value')

# Display sliders
display(function_dropdown,argument_slider)


Dropdown(description='Select Function:', options=('Monte Carlo Simulations', 'Response Surface Method', 'Artif…

IntSlider(value=25, description='Select Argument:', max=200, step=25)

### 1.2.3 The variable properties are selected.
The function is a lognormal distribution function with median and despersion are set from the text fields.

In [4]:
dspr_case = widgets.RadioButtons(options={'Standard Case':1, 'High Variance Case':2},value=1,description='The variance of the case study:')

ecu_med = widgets.BoundedFloatText(value=0.006,min=0.002,max=0.02,step=0.001,description='Ultimate concrete strain median:',disabled=False)
ecu_dspr = widgets.BoundedFloatText(value=0.2,min=0,max=1,step=0.01,description='Ultimate concrete strain dispersion:',disabled=False)
Vu_med = widgets.BoundedFloatText(value=170,min=50,max=1000,step=5,description='Ultimate shear strength median:',disabled=False)
Vu_dspr = widgets.BoundedFloatText(value=0.35,min=0,max=1,step=0.01,description='Ultimate shear strength dispersion:',disabled=False)

# Display BoundedFloatText widgets
display(ecu_med,ecu_dspr,Vu_med,Vu_dspr,dspr_case)

# Function to be called when any of the BoundedFloatText widgets change
def apply_single_function(change):
    global uniform_sample_set, ecu_list, Vult_list
    uniform_sample_set = sps.norm.cdf(sampling_set, 0, 1) # convert standard distribution to CDF
    ecu_list = sps.lognorm.ppf(uniform_sample_set[:,0],s=(np.log((1+ecu_dspr.value*dspr_case.value)**2)),scale=ecu_med.value) # scale back to log normal distribution
    Vult_list = sps.lognorm.ppf(uniform_sample_set[:,1],s=(np.log((1+Vu_dspr.value*dspr_case.value)**2)),scale=Vu_med.value) # scale back to log normal distribution
    print(f"Single Function is applied with arguments: ecu_median = {np.median(ecu_list)}")
    return ecu_list, Vult_list

# Attach the function application to the BoundedFloatText widgets' 'value' attribute
dspr_case.observe(apply_single_function, names='value')
ecu_med.observe(apply_single_function, names='value')
ecu_dspr.observe(apply_single_function, names='value')
Vu_med.observe(apply_single_function, names='value')
Vu_dspr.observe(apply_single_function, names='value')

def update_dspr_fact(change):
    global dspr_fact
    print(f"Dispersion factor has been changed to: {dspr_fact}x")

BoundedFloatText(value=0.006, description='Ultimate concrete strain median:', max=0.02, min=0.002, step=0.001)

BoundedFloatText(value=0.2, description='Ultimate concrete strain dispersion:', max=1.0, step=0.01)

BoundedFloatText(value=170.0, description='Ultimate shear strength median:', max=1000.0, min=50.0, step=5.0)

BoundedFloatText(value=0.35, description='Ultimate shear strength dispersion:', max=1.0, step=0.01)

RadioButtons(description='The variance of the case study:', options={'Standard Case': 1, 'High Variance Case':…

This is how the output file will be named based on the study case.

In [5]:
output_filename = "sims_"+"".join([word[0] for word in function_dropdown.value.split()])+"_var"+str(dspr_case.value)+"X"+"_"+str(argument_slider.value)+"_1.csv"
output_filename

'sims_MCS_var1X_25_1.csv'

### 1.2.4 The sampling dataset is rescaled to match the inserted properties.

In [6]:
uniform_sample_set = sps.norm.cdf(sampling_set, 0, 1) # convert standard distribution to CDF
ecu_list = sps.lognorm.ppf(uniform_sample_set[:,0],s=(np.log((1+ecu_dspr.value*dspr_case.value)**2)),scale=ecu_med.value) # scale back to log normal distribution
Vult_list = sps.lognorm.ppf(uniform_sample_set[:,1],s=(np.log((1+Vu_dspr.value*dspr_case.value)**2)),scale=Vu_med.value) # scale back to log normal distribution
ecu_list

array([0.00580689, 0.00589961, 0.00696957, 0.00354576, 0.00807228,
       0.00402866, 0.00418676, 0.00617898, 0.01258723, 0.00305353,
       0.00895596, 0.00478214, 0.00429725, 0.00413675, 0.00637352,
       0.0059107 , 0.00727364, 0.00623362, 0.01150684, 0.00597302,
       0.00902655, 0.00503783, 0.00718048, 0.00486834, 0.0033077 ])

## 1.3. Starting the nonlinear analysis

### 1.3.1 Downloading the ground motion record files
They are uploaded to my github profile in the folder "*Data_Input*".


In [7]:
#create directories
from pathlib import Path
Path("Data_Input").mkdir(parents=True, exist_ok=True)
Path("Data_Output").mkdir(parents=True, exist_ok=True)

# import strong motion records in "Data_Input" directory
!wget -O main.zip https://github.com/mtareqsoliman/msthesis/archive/main.zip && unzip -qq main.zip -d main && mv main/msthesis-main/Data_Input/* Data_Input/ && rm -rf main.zip main

--2024-01-28 22:02:17--  https://github.com/mtareqsoliman/msthesis/archive/main.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/mtareqsoliman/msthesis/zip/refs/heads/main [following]
--2024-01-28 22:02:17--  https://codeload.github.com/mtareqsoliman/msthesis/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 192.30.255.120
Connecting to codeload.github.com (codeload.github.com)|192.30.255.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [      <=>           ]  13.77M  10.8MB/s    in 1.3s    

2024-01-28 22:02:18 (10.8 MB/s) - ‘main.zip’ saved [14437858]



This is the record reader function that will prepare the extract the values from the ground motion file.
* nPts is number of points.
* dt is time step.

In [8]:
# record reader function

def ReadRecord (inFilename, outFilename):
    dt = 0.0
    npts = 0
    #inFileID = open(inFilename, 'r')
    inFileID = open(inFilename,'r')
    outFileID = open(outFilename, 'w')
    flag = 0
    for line in inFileID:
        if line == '\n':
            continue
        elif flag == 1:
            outFileID.write(line)
        else:
            words = line.split()
            lengthLine = len(words)
            if lengthLine >= 4:
                if words[0] == 'NPTS=':
                    for word in words:
                        if word != '':
                            if flag == 1:
                                dt = float(word)
                                break
                            if flag == 2:
                                npts = int(word.strip(','))
                                flag = 0
                            if word == 'DT=' or word == 'dt':
                                flag = 1
                            if word == 'NPTS=':
                                flag = 2
                elif words[-1] == 'DT':
                    count = 0
                    for word in words:
                        if word != '':
                            if count == 0:
                                npts = int(word)
                            elif count == 1:
                                dt = float(word)
                            elif word == 'DT':
                                flag = 1
                                break
                            count += 1
    inFileID.close()
    outFileID.close()
    return dt, npts

### 1.3.2 Applying the nonlinear time history analysis
The analysis will be repeated for each
* limit state combination
* ground motion record
* each record will be scaled from PGV= 0 [cm/s] to PGV= 40 [cm/s] with step size 5 [cm/s]

In [10]:
# =============================================================================
# Future improvement plans
# 1) get PGV from file data and make them unitless.
# 2) move the functions to another file, then import them.
# 3) allow to input the other structure properties and design.
# 4) completely make the design properties unitless.
# 5) curvature (np), identification of failure location
# 6) failure case (list + identify minimum value then column index then conc or steel)
# =============================================================================

import openseespy.opensees as ops
import opsvis as opsv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import itertools as itools
import subprocess as sp
import time
from google.colab import files
import base64
import requests

# function to calculate simulation time
start_time = time.time()
tmp = sp.call('cls',shell=True)


ecu_list = sps.lognorm.ppf(uniform_sample_set[:,0],s=(np.log((1+ecu_dspr.value*dspr_case.value)**2)),scale=ecu_med.value) # scale back to log normal distribution
Vult_list = sps.lognorm.ppf(uniform_sample_set[:,1],s=(np.log((1+Vu_dspr.value*dspr_case.value)**2)),scale=Vu_med.value) # scale back to log normal distribution


# list of record names to be used in the analysis with accelaration time series in units of [g]
record_list = ['RSN126_GAZLI_GAZ000','RSN461_MORGAN_HVR240','RSN518_PALMSPR_FVR045','RSN599_WHITTIER.A_A-FLO020','RSN721_SUPER.B_B-ICC090','RSN766_LOMAP_G02000','RSN828_CAPEMEND_PET000','RSN864_LANDERS_JOS090' ,'RSN1044_NORTHR_NWH360','RSN1148_KOCAELI_ARE000','RSN1197_CHICHI_CHY028-E','RSN1605_DUZCE_DZC270']
#record_list = ['RSN126_GAZLI_GAZ000']

pgv_list = [66.19,39.32,6.28,7.13,41.77,34.74,49.30,42.57,96.54,13.95,61.36,84.19] # PGV [cm/sec], manually calculated from
#pgv_list = [66.19]

# each record will be rescaled to be equivalent to PGV from 5 [cm/sec] to 40 [cm/sec]
factor_list = np.linspace(40,5,4)

#   results table     =  result - IM - record - ecu - Vult - Vb - ect - ecb - umax -uindex
ultimate_result = pd.DataFrame(np.zeros((len(factor_list)*len(record_list)*len(ecu_list),10)),columns=['state','IM','record','ecu','Vult','Vbmax','ect','ecb','uindex','convergence'])

j1 = 0 # row counter
nrows = len(factor_list)*len(record_list)*len(ecu_list) # results table rows

# results values in numpy array at each earthquake timestep   [lowerstrain BOT, upperstrain BOT,lowerstrain TOP, upperstrain TOP]
z_cstrn=np.zeros((18002,8,len(factor_list),len(record_list),len(ecu_list)))

# constants
g = 9.81
m = 1
kN = 1

for r1,record,pgv in zip(itools.count(),record_list,pgv_list):
    dt, nPts = ReadRecord('Data_Input/'+record+'.AT2', 'Data_Output/'+record+'.dat')
    for f1, factor in enumerate(factor_list):
        for e1,ecu,Vult in zip(itools.count(),ecu_list,Vult_list):
                print("==========================")
                print("Starting RCFrameGravity example")

                ops.wipe()
                ops.model('basic', '-ndm', 2, '-ndf', 3)

                bay,height1,height2 = 8.0,4.0,3.0

                ops.node(1, 0.0, 0.0)
                ops.node(2, bay, height1-height2)
                ops.node(3, 0.0, height1)
                ops.node(4, bay, height1)

                ops.fix(1, 1, 1, 1)
                ops.fix(2, 1, 1, 1)

                ops.equalDOF(3,4,1) # DOF of the beam ends are equal in horizontal direction

                fcu,Ec,ftc = [-30*10**3, 30*10**6,38340.0679] # fcu [kN/m2], Ec [kN/m2], ftc [kN/m2]
                fy, Es, bs = [400*10**3,200*10**6,0.03] # fy [kN/m2], Es [kN/m2], bs [unitless]
                ec0 = -0.002 # ec0 [unitless]

                ops.uniaxialMaterial('Concrete04', 1, fcu, ec0,  ecu,  Ec, ftc)
                #ops.uniaxialMaterial('Concrete04', 2, fcu, ec0,  ecu,  Ec, ftc)
                ops.uniaxialMaterial('Steel02', 3, fy, Es, bs)


                # column section generation
                seccol,bc,tc,cover,phic,ntopc,nbotc = [1,0.2,0.5,0.025,0.014,4,4]
                z1c, y1c, Asc = [bc/2,tc/2,math.pi*phic**2/4]
                fib_sec_col = [['section', 'Fiber', seccol, '-GJ', 0.0e6],
                            ['patch', 'rect', 1, 18, 1,-y1c,-z1c, y1c,z1c],
                            ['layer', 'straight', 3, ntopc, Asc, y1c - cover-phic/2, z1c - cover-phic/2, y1c - cover-phic/2, cover - z1c+phic/2],
                            ['layer', 'straight', 3, nbotc, Asc, cover - y1c+phic/2, z1c - cover-phic/2, cover - y1c+phic/2, cover - z1c+phic/2]]
                opsv.fib_sec_list_to_cmds(fib_sec_col)

                # beam end-section generation
                secbij,bb,tb,cover,phib,ntopbij,nbotbij = [2,0.2,0.7,0.025,0.016,4,2]
                z1b, y1b, Asb = [bb/2,tb/2,math.pi*phib**2/4]
                fib_sec_bij = [['section', 'Fiber', secbij, '-GJ', 0.0],
                            ['patch', 'rect', 1, 18, 1,-y1b,-z1b,y1b,z1b],
                            ['layer', 'straight', 3, ntopbij, Asb, y1b - cover-phib/2, z1b - cover-phib/2, y1b - cover-phib/2, cover - z1b+phib/2],
                            ['layer', 'straight', 3, nbotbij, Asb, cover - y1b+phib/2, z1b - cover-phib/2, cover - y1b+phib/2, cover - z1b+phib/2]]
                opsv.fib_sec_list_to_cmds(fib_sec_bij)

                # beam mid-section generation
                secbms,bb,tb,cover,phib,ntopbms,nbotbms = [3,0.2,0.7,0.025,0.016,2,4]
                z1b, y1b, Asb = [bb/2,tb/2,math.pi*phib**2/4]
                fib_sec_bms = [['section', 'Fiber', secbms, '-GJ', 0.0],
                            ['patch', 'rect', 1, 18, 1,-y1b,-z1b,y1b,z1b],
                            ['layer', 'straight', 3, ntopbms, Asb, y1b - cover-phib/2, z1b - cover-phib/2, y1b - cover-phib/2, cover - z1b+phib/2],
                            ['layer', 'straight', 3, nbotbms, Asb, cover - y1b+phib/2, z1b - cover-phib/2, cover - y1b+phib/2, cover - z1b+phib/2]]
                opsv.fib_sec_list_to_cmds(fib_sec_bms)

                ops.uniaxialMaterial('Elastic', 4,10**0)

                # restricting the axial deformation of the beam
                #section Aggregator $secTag $matTag1 $dof1 <-section $sectionTag>
                ops.section('Aggregator',20,       4,      'P', '-section',secbij)
                ops.section('Aggregator',30,       4,      'P', '-section',secbms)

                ops.geomTransf('Linear', 1)

                # plastic hinge length at the two columns
                #plc = 0.08*L[m]       + 0.022*phi[m]*fy[MPa]
                plc1=(0.08*0.38*height1)+(0.022*phic*fy*10**-3)
                plc2=(0.08*0.22*height2)+(0.022*phic*fy*10**-3)
                ops.beamIntegration('HingeRadau', 1, 1, plc1,1,plc1,1)
                ops.beamIntegration('HingeRadau', 2, 1, plc2,1,plc2,1)

                #               e          tag ndI ndJ transfTag integrationTag
                ops.element('forceBeamColumn', 1,   1,  3,    1,           1)
                ops.element('forceBeamColumn', 2,   2,  4,    1,           2)

                ops.geomTransf('Linear', 2)

                # plastic hinge length at beam midsection
                plbm=(0.08*0.18*bay)+(0.022*phib*fy*10**-3)

                ops.beamIntegration('HingeRadau', 3, 2, plbm,2,plbm,30)
                #               e             tag  ndI ndJ transfTag integrationTag
                ops.element('forceBeamColumn', 3,   3,  4,    2,           3)

                ops.timeSeries('Linear', 1)
                ops.pattern('Plain', 1, 1)


                P, Wz = 500.0, 36.0 # P concentrated load in the middle [kN], Wz distributed load [kN/m]
                #       nd  FX, FY, MZ
                ops.load(3, 0.0, -P, 0.0) # kN
                ops.load(4, 0.0, -P, 0.0) # kN
                ops.eleLoad('-ele', 3, '-type', '-beamUniform', -Wz) # kN/m

                n= Es/Ec # modular ratio

                # moment of inertia of the column
                colIz = (0.2*0.5**3/12)+(n-1)*(nbotc*Asc)*(z1c-cover-phic/2)**2+(n-1)*(ntopc*Asc)*(z1c-cover-phic/2)**2

                # lateral stiffness
                Kl= 12*Ec*colIz*(1/(height1**3)+1/(height2**3))

                # time period of the structure (given in the problem)
                Tn = 0.48

                # calculating mass of the structure
                massl = ((Tn**2)*Kl)/(4*(math.pi**2))


                # node#,     mx   my  mz
                ops.mass(3, massl/4, 0, 0.0)
                ops.mass(4, massl/4, 0, 0.0)

                ops.system('BandGeneral')
                ops.constraints('Transformation')
                ops.numberer('RCM')
                ops.test('NormDispIncr', 1.0e-12, 10, 3)
                ops.algorithm('Newton')
                ops.integrator('LoadControl', 0.1)
                ops.analysis('Static')

                # analyzing the dead load case in 10 steps
                ops.analyze(10)

                print("==========================")

                # =============================================================================
                #
                print("Start RCFrameEarthquake Example")
                #
                # =============================================================================

                ops.wipeAnalysis()
                ops.loadConst('-time', 0.0)

                ops.timeSeries('Path', 2, '-filePath', 'Data_Output/'+record+'.dat', '-dt', dt, '-factor', factor*g/(pgv))
                ops.pattern('UniformExcitation',  2,   1,  '-accel', 2)
                #ops.rayleigh(0.0, 0.0, 0.0, 0.000625)
                ops.system('BandGeneral')
                ops.constraints('Plain')
                ops.test('NormDispIncr', 1.0e-12,  10 )
                ops.algorithm('Newton')
                ops.numberer('RCM')
                ops.integrator('Newmark',  0.5,  0.25 )
                ops.analysis('Transient')
                tFinal, tCurrent, ok = nPts*dt, ops.getTime(), 0
                ops.record()

                i = 0
                z_v1 = []
                z_v2 = []

                while ok == 0 and tCurrent < tFinal:

                    tCurrent = ops.getTime()

                    z_v1.append(abs(ops.eleForce(1,1))),z_v2.append(abs(ops.eleForce(2,1)))
                    # [BOT lowerstrain, BOT upperconc strain,TOP lowerstrain, TOP upperstrain]
                    z_cstrn[i,0,f1,r1,e1],z_cstrn[i,1,f1,r1,e1],z_cstrn[i,2,f1,r1,e1],z_cstrn[i,3,f1,r1,e1]=ops.eleResponse(1, 'section', 1, 'fiber', -y1c, 0, 1, 'stressStrain')[1],ops.eleResponse(1, 'section', 1, 'fiber', y1c, 0, 1, 'stressStrain')[1],ops.eleResponse(1, 'section', 6, 'fiber', -y1c, 0, 1, 'stressStrain')[1],ops.eleResponse(1, 'section', 6, 'fiber', y1c, 0, 1, 'stressStrain')[1]
                    z_cstrn[i,4,f1,r1,e1],z_cstrn[i,5,f1,r1,e1],z_cstrn[i,6,f1,r1,e1],z_cstrn[i,7,f1,r1,e1]=ops.eleResponse(2, 'section', 1, 'fiber', -y1c, 0, 1, 'stressStrain')[1],ops.eleResponse(2, 'section', 1, 'fiber', y1c, 0, 1, 'stressStrain')[1],ops.eleResponse(2, 'section', 6, 'fiber', -y1c, 0, 1, 'stressStrain')[1],ops.eleResponse(2, 'section', 6, 'fiber', y1c, 0, 1, 'stressStrain')[1]

                    if ultimate_result.loc[j1,'state'] == False and min(z_cstrn[i,:,f1,r1,e1]) < ecu:
                        ultimate_result.loc[j1,'uindex'] = i
                        ultimate_result.loc[j1,'state'] = 1
                    if ultimate_result.loc[j1,'state'] == False and max(max(z_v1),max(z_v2),abs(min(z_v1)),abs(min(z_v2))) > Vult:
                        ultimate_result.loc[j1,'uindex'] = i
                        ultimate_result.loc[j1,'state'] = 1

                    ok = ops.analyze(1, dt)

                    if ok != 0:
                        print("modified newton failed")
                        print("regular newton failed .. lets try an initail stiffness for this step")
                        ops.test('NormDispIncr', 1.0e-12,  100, 0)
                        ops.algorithm('ModifiedNewton', '-initial')
                        ok =ops.analyze( 1, dt)
                        if ok == 0:
                            print('Succeded, returning back to Newton Algorithm')
                            ops.algorithm('Newton')
                    if ok != 0:
                        print("Trying Newton with Initial Tangent")
                        ops.test("EnergyIncr",1.e-8,6,0)
                        ops.algorithm("Newton", '-initial')
                        ok =ops.analyze( 1, dt)
                        if ok == 0:
                            print('Succeded, returning back to Newton Algorithm')
                            ops.test("NormDispIncr", 1.e-8,2000,0)
                            ops.algorithm('Newton')
                    if ok!=0:
                        print("Trying Broyden ..")
                        ops.algorithm("Broyden", 8)
                        ok =ops.analyze( 1, dt)
                        if ok == 0:
                            print('Succeded, returning back to Newton Algorithm')
                            ops.algorithm('Newton')
                    if ok!=0:
                        print("Trying NewtonWithLineSearch")
                        ops.algorithm("NewtonLineSearch",0.8)
                        ok =ops.analyze( 1, dt)
                        if ok == 0:
                            print('Succeded, returning back to Newton Algorithm')
                            ops.algorithm('Newton')
                    if ok!=0:
                        print('Failed to converge :( breaking the loop')
                        ultimate_result.loc[j1,'convergence'] = 'FAILED'
                        break

                    i = i+1

                ops.wipe()

                # =============================================================================
                # Postprocessing Stage
                # =============================================================================
                print("End of time history analysis, Postprocessing has started....")
                print("Progress ="+str(round((j1*100/nrows),2))+" %")

                            # 0result - 1IM - 2record - 3ecu - 4Vult - 5Vbmax - 6ect - 7ecb - 8umax -9uindex

                ultimate_result.loc[j1,'IM'] = factor  #IM
                ultimate_result.loc[j1,'record'] = record  #record
                ultimate_result.loc[j1,'ecu'] = ecu     #ecu
                ultimate_result.loc[j1,'Vult'] = Vult    #V

                #ultimate_result.loc[j1,'umax'] = max(abs(min(z_u4x[:,0,f1,r1,e1])),max(z_u4x[:,0,f1,r1,e1]))
                ultimate_result.loc[j1,'ect'] = min(min(z_cstrn[:,6,f1,r1,e1]),min(z_cstrn[:,7,f1,r1,e1]))
                ultimate_result.loc[j1,'ecb'] = min(min(z_cstrn[:,4,f1,r1,e1]),min(z_cstrn[:,5,f1,r1,e1]))
                ultimate_result.loc[j1,'Vbmax'] = max(z_v1+z_v2)


                # result - IM - record - ecu - Vult - Vb - ect - ecb - umax

                j1= j1+1

        def graphPlotter(x,y,lbl,ttl,xlbl,ylbl):
            fig, ax = plt.subplots(figsize=(11,7))
            ax.plot(x, y, label=lbl)


print("--- %s seconds ---" % (time.time() - start_time))
output_filename = "sims_"+"".join([word[0] for word in function_dropdown.value.split()])+"_var"+str(dspr_case.value)+"X"+"_"+str(argument_slider.value)+"_1.csv"
ultimate_result.to_csv(output_filename,index=False)
files.download(output_filename)

Starting RCFrameGravity example
Start RCFrameEarthquake Example


after: 10 iterations  current Norm: 0.000325969 (max: 1e-12, Norm deltaR: 7.32177)
NewtonRaphson::solveCurrentStep() -the ConvergenceTest object failed in test()
DirectIntegrationAnalysis::analyze() - the Algorithm failed at time 8.1774
OpenSees > analyze failed, returned: -3 error flag
after: 100 iterations  current Norm: 4.6286e-08 (max: 1e-12, Norm deltaR: 0.0227071)
ModifiedNewton::solveCurrentStep() -the ConvergenceTest object failed in test()
DirectIntegrationAnalysis::analyze() - the Algorithm failed at time 8.1774
OpenSees > analyze failed, returned: -3 error flag
after: 6 iterations
 current EnergyIncr: 2.99074e-05 (max: 1e-08) 	Norm deltaX: 1.07308e-05, Norm deltaR: 6.42839
NewtonRaphson::solveCurrentStep() -the ConvergenceTest object failed in test()
DirectIntegrationAnalysis::analyze() - the Algorithm failed at time 8.1774
OpenSees > analyze failed, returned: -3 error flag
after: 10 iterations
 current EnergyIncr: 6.31329e-05 (max: 1e-08) 	Norm deltaX: 0.000336199, Norm del

modified newton failed
regular newton failed .. lets try an initail stiffness for this step
Trying Newton with Initial Tangent
Trying Broyden ..
Succeded, returning back to Newton Algorithm


after: 6 iterations
 current EnergyIncr: 16.0339 (max: 1e-08) 	Norm deltaX: 0.0445315, Norm deltaR: 832.179
NewtonRaphson::solveCurrentStep() -the ConvergenceTest object failed in test()
DirectIntegrationAnalysis::analyze() - the Algorithm failed at time 11.1804
OpenSees > analyze failed, returned: -3 error flag
after: 100 iterations  current Norm: 1.28221e-05 (max: 1e-12, Norm deltaR: 3.09024)
ModifiedNewton::solveCurrentStep() -the ConvergenceTest object failed in test()
DirectIntegrationAnalysis::analyze() - the Algorithm failed at time 11.1804
OpenSees > analyze failed, returned: -3 error flag
after: 6 iterations
 current EnergyIncr: 0.0270898 (max: 1e-08) 	Norm deltaX: 0.000489765, Norm deltaR: 154.316
NewtonRaphson::solveCurrentStep() -the ConvergenceTest object failed in test()
DirectIntegrationAnalysis::analyze() - the Algorithm failed at time 11.1804
OpenSees > analyze failed, returned: -3 error flag
after: 10 iterations
 current EnergyIncr: 0.00511213 (max: 1e-08) 	Norm delta

modified newton failed
regular newton failed .. lets try an initail stiffness for this step
Trying Newton with Initial Tangent
Trying Broyden ..
Succeded, returning back to Newton Algorithm
End of time history analysis, Postprocessing has started....
Progress =0.0 %
Starting RCFrameGravity example
Start RCFrameEarthquake Example


...

modified newton failed
regular newton failed .. lets try an initail stiffness for this step
Succeded, returning back to Newton Algorithm
End of time history analysis, Postprocessing has started....
Progress =72.0 %
Starting RCFrameGravity example
Start RCFrameEarthquake Example


after: 10 iterations  current Norm: 1.32506e-12 (max: 1e-12, Norm deltaR: 3.54778e-13)
NewtonRaphson::solveCurrentStep() -the ConvergenceTest object failed in test()
DirectIntegrationAnalysis::analyze() - the Algorithm failed at time 7.2336
OpenSees > analyze failed, returned: -3 error flag


modified newton failed
regular newton failed .. lets try an initail stiffness for this step
Succeded, returning back to Newton Algorithm
End of time history analysis, Postprocessing has started....
Progress =73.0 %
Starting RCFrameGravity example
Start RCFrameEarthquake Example
End of time history analysis, Postprocessing has started....
Progress =74.0 %
Starting RCFrameGravity example
Start RCFrameEarthquake Example
End of time history analysis, Postprocessing has started....
Progress =75.0 %
Starting RCFrameGravity example
Start RCFrameEarthquake Example
End of time history analysis, Postprocessing has started....
Progress =76.0 %
Starting RCFrameGravity example
Start RCFrameEarthquake Example
End of time history analysis, Postprocessing has started....
Progress =77.0 %
Starting RCFrameGravity example
Start RCFrameEarthquake Example
End of time history analysis, Postprocessing has started....
Progress =78.0 %
Starting RCFrameGravity example
Start RCFrameEarthquake Example
End of time

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>